# üìä Visualizaci√≥n de Datos en Ingenier√≠a de Datos

## Objetivos de Aprendizaje

Al finalizar este notebook, ser√°s capaz de:

1. ‚úÖ Crear visualizaciones efectivas con **Matplotlib**
2. ‚úÖ Usar **Seaborn** para an√°lisis estad√≠stico visual
3. ‚úÖ Generar gr√°ficos interactivos con **Plotly**
4. ‚úÖ Dise√±ar dashboards b√°sicos
5. ‚úÖ Aplicar mejores pr√°cticas de visualizaci√≥n de datos

---

## 1. Configuraci√≥n Inicial

In [ ]:
# Importaciones necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')

# Configuraci√≥n de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configuraci√≥n de tama√±o de figuras por defecto
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("‚úÖ Librer√≠as cargadas correctamente")

## 2. Carga de Datos

In [ ]:
# Cargar datasets
productos = pd.read_csv('../../datasets/raw/productos.csv')
clientes = pd.read_csv('../../datasets/raw/clientes.csv')
ventas = pd.read_csv('../../datasets/raw/ventas.csv')

# Convertir fechas
ventas['fecha_venta'] = pd.to_datetime(ventas['fecha_venta'])
clientes['fecha_registro'] = pd.to_datetime(clientes['fecha_registro'])

print(f"üì¶ Productos: {len(productos)} registros")
print(f"üë• Clientes: {len(clientes)} registros")
print(f"üí∞ Ventas: {len(ventas)} registros")

# Vista previa
display(ventas.head())

## 3. Matplotlib: Gr√°ficos B√°sicos

Matplotlib es la biblioteca fundamental de visualizaci√≥n en Python.

### 3.1 Gr√°fico de L√≠neas - Evoluci√≥n de Ventas

In [ ]:
# Agrupar ventas por fecha
ventas_diarias = ventas.groupby('fecha_venta')['total'].sum().reset_index()

# Crear gr√°fico
plt.figure(figsize=(14, 6))
plt.plot(ventas_diarias['fecha_venta'], ventas_diarias['total'], 
         marker='o', linestyle='-', linewidth=2, markersize=6, color='#2E86AB')

# Personalizaci√≥n
plt.title('üìà Evoluci√≥n de Ventas Diarias', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Total Ventas ($)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Agregar l√≠nea de promedio
promedio = ventas_diarias['total'].mean()
plt.axhline(y=promedio, color='red', linestyle='--', linewidth=2, alpha=0.7, label=f'Promedio: ${promedio:.2f}')
plt.legend()

plt.show()

print(f"üíµ Venta promedio diaria: ${promedio:.2f}")
print(f"üìä Venta m√°xima: ${ventas_diarias['total'].max():.2f}")
print(f"üìâ Venta m√≠nima: ${ventas_diarias['total'].min():.2f}")

### 3.2 Gr√°fico de Barras - Ventas por Categor√≠a

In [ ]:
# Merge para obtener categor√≠as
ventas_productos = ventas.merge(productos, left_on='producto_id', right_on='producto')

# Agrupar por categor√≠a
ventas_categoria = ventas_productos.groupby('categoria')['total'].sum().sort_values(ascending=True)

# Crear gr√°fico horizontal
plt.figure(figsize=(12, 8))
colores = plt.cm.viridis(np.linspace(0, 1, len(ventas_categoria)))
bars = plt.barh(ventas_categoria.index, ventas_categoria.values, color=colores)

# Agregar valores en las barras
for i, bar in enumerate(bars):
    width = bar.get_width()
    plt.text(width, bar.get_y() + bar.get_height()/2, 
             f'${width:.2f}', 
             ha='left', va='center', fontweight='bold', fontsize=10)

plt.title('üíº Ventas Totales por Categor√≠a de Producto', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Total Ventas ($)', fontsize=12)
plt.ylabel('Categor√≠a', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nüèÜ Top 3 Categor√≠as:")
for i, (cat, total) in enumerate(ventas_categoria.tail(3).items(), 1):
    print(f"{i}. {cat}: ${total:.2f}")

### 3.3 Gr√°fico de Pastel - Distribuci√≥n de M√©todos de Pago

In [ ]:
# Contar m√©todos de pago
metodos_pago = ventas['metodo_pago'].value_counts()

# Crear gr√°fico de pastel
plt.figure(figsize=(10, 10))
colores = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
explode = [0.05 if i == 0 else 0 for i in range(len(metodos_pago))]

plt.pie(metodos_pago.values, 
        labels=metodos_pago.index, 
        autopct='%1.1f%%',
        startangle=90,
        colors=colores,
        explode=explode,
        shadow=True,
        textprops={'fontsize': 11, 'fontweight': 'bold'})

plt.title('üí≥ Distribuci√≥n de M√©todos de Pago', fontsize=16, fontweight='bold', pad=20)
plt.axis('equal')
plt.tight_layout()
plt.show()

print("\nüìä Desglose de m√©todos de pago:")
for metodo, cantidad in metodos_pago.items():
    porcentaje = (cantidad / len(ventas)) * 100
    print(f"  ‚Ä¢ {metodo}: {cantidad} transacciones ({porcentaje:.1f}%)")

### 3.4 Subplots - M√∫ltiples Gr√°ficos

In [ ]:
# Crear figura con 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('üìä Dashboard de An√°lisis de Ventas', fontsize=18, fontweight='bold', y=1.00)

# 1. Distribuci√≥n de precios
axes[0, 0].hist(productos['precio'], bins=15, color='#6C5CE7', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribuci√≥n de Precios de Productos', fontweight='bold')
axes[0, 0].set_xlabel('Precio ($)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].grid(alpha=0.3)

# 2. Stock por categor√≠a
stock_categoria = productos.groupby('categoria')['stock'].sum().sort_values()
axes[0, 1].barh(stock_categoria.index, stock_categoria.values, color='#00B894')
axes[0, 1].set_title('Stock Total por Categor√≠a', fontweight='bold')
axes[0, 1].set_xlabel('Unidades en Stock')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Clientes por segmento
segmentos = clientes['segmento'].value_counts()
axes[1, 0].pie(segmentos.values, labels=segmentos.index, autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Distribuci√≥n de Clientes por Segmento', fontweight='bold')

# 4. Ventas por estado
ventas_estado = ventas.groupby('estado')['total'].sum()
axes[1, 1].bar(ventas_estado.index, ventas_estado.values, color=['#00D2D3', '#55EFC4', '#FD79A8'])
axes[1, 1].set_title('Ventas Totales por Estado', fontweight='bold')
axes[1, 1].set_xlabel('Estado')
axes[1, 1].set_ylabel('Total ($)')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Seaborn: Visualizaci√≥n Estad√≠stica

Seaborn facilita la creaci√≥n de gr√°ficos estad√≠sticos complejos.

### 4.1 Box Plot - An√°lisis de Distribuci√≥n

In [ ]:
plt.figure(figsize=(14, 7))
sns.boxplot(data=ventas_productos, x='categoria', y='total', palette='Set2')
plt.title('üì¶ Distribuci√≥n de Ventas por Categor√≠a (Box Plot)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Categor√≠a', fontsize=12)
plt.ylabel('Total de Venta ($)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nüìä Estad√≠sticas por categor√≠a:")
print(ventas_productos.groupby('categoria')['total'].describe()[['mean', '50%', 'std']].round(2))

### 4.2 Violin Plot - Densidad y Distribuci√≥n

In [ ]:
plt.figure(figsize=(14, 7))
sns.violinplot(data=ventas, x='metodo_pago', y='total', palette='muted', inner='quartile')
plt.title('üéª Distribuci√≥n de Montos por M√©todo de Pago', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('M√©todo de Pago', fontsize=12)
plt.ylabel('Total ($)', fontsize=12)
plt.xticks(rotation=30)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.3 Heatmap - Matriz de Correlaci√≥n

In [ ]:
# Preparar datos para correlaci√≥n
ventas_productos['mes'] = ventas_productos['fecha_venta'].dt.month
ventas_productos['dia_semana'] = ventas_productos['fecha_venta'].dt.dayofweek

# Seleccionar columnas num√©ricas
columnas_numericas = ['cantidad', 'precio_unitario', 'total', 'precio', 'stock', 'mes', 'dia_semana']
correlacion = ventas_productos[columnas_numericas].corr()

# Crear heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlacion, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('üî• Matriz de Correlaci√≥n - Variables de Ventas', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nüîç Correlaciones m√°s fuertes:")
# Obtener correlaciones sin la diagonal
corr_values = correlacion.abs().unstack()
corr_values = corr_values[corr_values < 1]
print(corr_values.sort_values(ascending=False).head(5))

### 4.4 Pair Plot - Relaciones entre Variables

In [ ]:
# Crear subset de datos
datos_pairplot = ventas_productos[['cantidad', 'precio_unitario', 'total', 'estado']].copy()

# Crear pairplot
sns.pairplot(datos_pairplot, hue='estado', palette='Set1', diag_kind='kde', corner=True)
plt.suptitle('üìä An√°lisis de Relaciones entre Variables', y=1.02, fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### 4.5 Count Plot - Frecuencia de Categor√≠as

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=clientes, x='segmento', hue='pais', palette='pastel')
plt.title('üë• Distribuci√≥n de Clientes por Segmento y Pa√≠s', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Segmento', fontsize=12)
plt.ylabel('Cantidad de Clientes', fontsize=12)
plt.legend(title='Pa√≠s', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Plotly: Gr√°ficos Interactivos

Plotly permite crear visualizaciones interactivas para dashboards web.

### 5.1 Scatter Plot Interactivo

In [ ]:
fig = px.scatter(ventas_productos, 
                 x='precio', 
                 y='total',
                 size='cantidad',
                 color='categoria',
                 hover_data=['producto', 'metodo_pago'],
                 title='üíé Relaci√≥n Precio vs Total de Venta (Interactivo)',
                 labels={'precio': 'Precio Unitario ($)', 'total': 'Total de Venta ($)'},
                 template='plotly_white')

fig.update_layout(height=600, font=dict(size=12))
fig.show()

print("üí° Tip: Pasa el mouse sobre los puntos para ver detalles")

### 5.2 Gr√°fico de L√≠neas con Animaci√≥n

In [ ]:
# Preparar datos para serie temporal
ventas_diarias_completo = ventas.copy()
ventas_diarias_completo['mes'] = ventas_diarias_completo['fecha_venta'].dt.to_period('M').astype(str)
ventas_diarias_completo = ventas_diarias_completo.sort_values('fecha_venta')

fig = px.line(ventas_diarias_completo, 
              x='fecha_venta', 
              y='total',
              color='metodo_pago',
              title='üìà Evoluci√≥n de Ventas por M√©todo de Pago',
              labels={'fecha_venta': 'Fecha', 'total': 'Total ($)', 'metodo_pago': 'M√©todo de Pago'},
              template='plotly_dark')

fig.update_traces(mode='lines+markers')
fig.update_layout(height=600, hovermode='x unified')
fig.show()

### 5.3 Gr√°fico de Barras Apiladas

In [ ]:
# Agrupar por categor√≠a y estado
ventas_cat_estado = ventas_productos.groupby(['categoria', 'estado'])['total'].sum().reset_index()

fig = px.bar(ventas_cat_estado, 
             x='categoria', 
             y='total',
             color='estado',
             title='üìä Ventas por Categor√≠a y Estado (Apiladas)',
             labels={'categoria': 'Categor√≠a', 'total': 'Total Ventas ($)', 'estado': 'Estado'},
             template='seaborn',
             barmode='stack')

fig.update_layout(height=600, xaxis_tickangle=-45)
fig.show()

### 5.4 Sunburst Chart - Jerarqu√≠a de Ventas

In [ ]:
# Preparar datos jer√°rquicos
jerarquia_ventas = ventas_productos.groupby(['categoria', 'producto'])['total'].sum().reset_index()

fig = px.sunburst(jerarquia_ventas, 
                  path=['categoria', 'producto'], 
                  values='total',
                  title='‚òÄÔ∏è Jerarqu√≠a de Ventas: Categor√≠a ‚Üí Producto',
                  template='ggplot2')

fig.update_layout(height=700)
fig.show()

print("üí° Tip: Haz clic en las secciones para explorar la jerarqu√≠a")

### 5.5 Dashboard con M√∫ltiples Gr√°ficos

In [ ]:
# Crear subplot con 4 gr√°ficos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Ventas Diarias', 'M√©todos de Pago', 'Top Productos', 'Ventas por Estado'),
    specs=[[{'type': 'scatter'}, {'type': 'pie'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# 1. L√≠nea de ventas diarias
ventas_dia = ventas.groupby('fecha_venta')['total'].sum().reset_index()
fig.add_trace(
    go.Scatter(x=ventas_dia['fecha_venta'], y=ventas_dia['total'], mode='lines+markers', name='Ventas'),
    row=1, col=1
)

# 2. Pie chart de m√©todos de pago
metodos = ventas['metodo_pago'].value_counts()
fig.add_trace(
    go.Pie(labels=metodos.index, values=metodos.values, name='M√©todos'),
    row=1, col=2
)

# 3. Top productos
top_productos = ventas_productos.groupby('producto')['total'].sum().nlargest(5)
fig.add_trace(
    go.Bar(x=top_productos.values, y=top_productos.index, orientation='h', name='Top Productos'),
    row=2, col=1
)

# 4. Ventas por estado
ventas_estado = ventas.groupby('estado')['total'].sum()
fig.add_trace(
    go.Bar(x=ventas_estado.index, y=ventas_estado.values, name='Estados'),
    row=2, col=2
)

# Actualizar layout
fig.update_layout(
    title_text='üìä Dashboard Ejecutivo de Ventas',
    title_font_size=20,
    showlegend=False,
    height=800,
    template='plotly_white'
)

fig.show()

print("‚úÖ Dashboard interactivo generado correctamente")

## 6. Mejores Pr√°cticas de Visualizaci√≥n

### 6.1 Principios de Dise√±o

In [ ]:
print("""
‚úÖ MEJORES PR√ÅCTICAS PARA VISUALIZACI√ìN DE DATOS

1. üéØ CLARIDAD
   ‚Ä¢ T√≠tulo descriptivo y claro
   ‚Ä¢ Etiquetas en los ejes
   ‚Ä¢ Leyendas cuando sea necesario
   ‚Ä¢ Unidades expl√≠citas

2. üé® COLOR
   ‚Ä¢ Usar paletas accesibles (color-blind friendly)
   ‚Ä¢ M√°ximo 5-7 colores diferentes
   ‚Ä¢ Colores consistentes entre gr√°ficos
   ‚Ä¢ Alto contraste para legibilidad

3. üìä TIPO DE GR√ÅFICO ADECUADO
   ‚Ä¢ L√≠neas: Tendencias temporales
   ‚Ä¢ Barras: Comparaciones entre categor√≠as
   ‚Ä¢ Scatter: Relaciones entre variables
   ‚Ä¢ Pie: Proporciones (m√°ximo 5 categor√≠as)
   ‚Ä¢ Heatmap: Matrices y correlaciones

4. üìè ESCALAS
   ‚Ä¢ Empezar eje Y en cero (barras)
   ‚Ä¢ Escala logar√≠tmica para rangos amplios
   ‚Ä¢ Evitar distorsiones

5. üìù ANOTACIONES
   ‚Ä¢ Resaltar puntos importantes
   ‚Ä¢ Agregar contexto cuando sea necesario
   ‚Ä¢ L√≠neas de referencia (promedios, metas)

6. üé≠ SIMPLICIDAD
   ‚Ä¢ Eliminar elementos innecesarios
   ‚Ä¢ Una idea principal por gr√°fico
   ‚Ä¢ Evitar 3D sin motivo
   ‚Ä¢ Reducir chartjunk

7. üì± INTERACTIVIDAD (Web)
   ‚Ä¢ Tooltips informativos
   ‚Ä¢ Zoom y pan cuando sea √∫til
   ‚Ä¢ Filtros din√°micos
   ‚Ä¢ Exportaci√≥n de datos
""")

### 6.2 Comparaci√≥n: Gr√°fico Malo vs Bueno

In [ ]:
# Datos de ejemplo
ventas_mes = ventas_productos.groupby(ventas_productos['fecha_venta'].dt.to_period('M').astype(str))['total'].sum()

# MALO: Sin contexto, colores inapropiados, sin etiquetas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gr√°fico MALO
ax1.bar(range(len(ventas_mes)), ventas_mes.values, color='red')
ax1.set_title('Ventas')
ax1.text(0.5, 0.95, '‚ùå MALO', transform=ax1.transAxes, 
         fontsize=16, fontweight='bold', color='red', ha='center')

# Gr√°fico BUENO
ax2.bar(ventas_mes.index, ventas_mes.values, color='#2E86AB', edgecolor='black', alpha=0.8)
ax2.set_title('üìä Evoluci√≥n de Ventas Mensuales', fontsize=14, fontweight='bold', pad=15)
ax2.set_xlabel('Mes', fontsize=11)
ax2.set_ylabel('Total Ventas ($)', fontsize=11)
ax2.grid(axis='y', alpha=0.3)
ax2.axhline(y=ventas_mes.mean(), color='red', linestyle='--', alpha=0.7, label='Promedio')

# Agregar valores en las barras
for i, v in enumerate(ventas_mes.values):
    ax2.text(i, v, f'${v:.0f}', ha='center', va='bottom', fontweight='bold')

ax2.legend()
ax2.text(0.5, 0.95, '‚úÖ BUENO', transform=ax2.transAxes, 
         fontsize=16, fontweight='bold', color='green', ha='center')

plt.tight_layout()
plt.show()

print("""
üîç Diferencias clave:

‚ùå MALO:
  ‚Ä¢ Sin etiquetas en ejes
  ‚Ä¢ T√≠tulo gen√©rico
  ‚Ä¢ Color agresivo sin contraste
  ‚Ä¢ Sin contexto (promedio, valores)
  ‚Ä¢ Sin grid para referencia

‚úÖ BUENO:
  ‚Ä¢ T√≠tulo descriptivo con emoji
  ‚Ä¢ Etiquetas claras con unidades
  ‚Ä¢ Color profesional con contraste
  ‚Ä¢ L√≠nea de promedio para contexto
  ‚Ä¢ Grid para facilitar lectura
  ‚Ä¢ Valores anotados
""")

## 7. Caso Pr√°ctico: Dashboard Completo

In [ ]:
# Funci√≥n para crear dashboard profesional
def crear_dashboard_ventas(ventas_df, productos_df, clientes_df):
    """
    Crea un dashboard completo de an√°lisis de ventas.
    """
    # Merge de datos
    datos_completos = ventas_df.merge(
        productos_df, left_on='producto_id', right_on='producto'
    ).merge(
        clientes_df, left_on='cliente_id', right_on='cliente_id'
    )
    
    # Crear figura
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # 1. KPIs principales (arriba)
    ax_kpi = fig.add_subplot(gs[0, :])
    ax_kpi.axis('off')
    
    # Calcular KPIs
    total_ventas = datos_completos['total'].sum()
    num_transacciones = len(datos_completos)
    ticket_promedio = total_ventas / num_transacciones
    num_clientes = datos_completos['cliente_id'].nunique()
    
    kpis = [
        ('üí∞ Total Ventas', f'${total_ventas:,.2f}'),
        ('üõí Transacciones', f'{num_transacciones:,}'),
        ('üìä Ticket Promedio', f'${ticket_promedio:.2f}'),
        ('üë• Clientes √önicos', f'{num_clientes:,}')
    ]
    
    for i, (titulo, valor) in enumerate(kpis):
        x_pos = 0.125 + i * 0.25
        ax_kpi.text(x_pos, 0.7, titulo, ha='center', fontsize=12, fontweight='bold')
        ax_kpi.text(x_pos, 0.3, valor, ha='center', fontsize=18, fontweight='bold', color='#2E86AB')
    
    # 2. Evoluci√≥n temporal
    ax1 = fig.add_subplot(gs[1, :])
    ventas_diarias = datos_completos.groupby('fecha_venta')['total'].sum()
    ax1.plot(ventas_diarias.index, ventas_diarias.values, marker='o', linewidth=2, markersize=6)
    ax1.fill_between(ventas_diarias.index, ventas_diarias.values, alpha=0.3)
    ax1.set_title('üìà Evoluci√≥n Diaria de Ventas', fontsize=14, fontweight='bold', pad=15)
    ax1.set_xlabel('Fecha')
    ax1.set_ylabel('Total ($)')
    ax1.grid(alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # 3. Top productos
    ax2 = fig.add_subplot(gs[2, 0])
    top_prods = datos_completos.groupby('producto')['total'].sum().nlargest(5)
    ax2.barh(top_prods.index, top_prods.values, color='#6C5CE7')
    ax2.set_title('üèÜ Top 5 Productos', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Ventas ($)')
    ax2.grid(axis='x', alpha=0.3)
    
    # 4. Segmentos de clientes
    ax3 = fig.add_subplot(gs[2, 1])
    segmentos_ventas = datos_completos.groupby('segmento')['total'].sum()
    ax3.pie(segmentos_ventas.values, labels=segmentos_ventas.index, autopct='%1.1f%%', startangle=90)
    ax3.set_title('üë• Ventas por Segmento', fontsize=12, fontweight='bold')
    
    # 5. M√©todos de pago
    ax4 = fig.add_subplot(gs[2, 2])
    metodos = datos_completos['metodo_pago'].value_counts()
    ax4.bar(metodos.index, metodos.values, color='#00B894')
    ax4.set_title('üí≥ M√©todos de Pago', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Frecuencia')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(axis='y', alpha=0.3)
    
    # T√≠tulo principal
    fig.suptitle('üìä DASHBOARD DE AN√ÅLISIS DE VENTAS', 
                 fontsize=20, fontweight='bold', y=0.98)
    
    plt.show()
    
    # Resumen estad√≠stico
    print("\n" + "="*60)
    print("üìä RESUMEN EJECUTIVO")
    print("="*60)
    print(f"\nüí∞ Ventas Totales: ${total_ventas:,.2f}")
    print(f"üõí Total Transacciones: {num_transacciones:,}")
    print(f"üìä Ticket Promedio: ${ticket_promedio:.2f}")
    print(f"üë• Clientes √önicos: {num_clientes:,}")
    print(f"\nüèÜ Producto m√°s vendido: {top_prods.index[0]} (${top_prods.values[0]:.2f})")
    print(f"üí≥ M√©todo de pago preferido: {metodos.index[0]} ({metodos.values[0]} transacciones)")
    print(f"üëî Segmento con m√°s ventas: {segmentos_ventas.idxmax()} (${segmentos_ventas.max():.2f})")
    print("\n" + "="*60)

# Ejecutar dashboard
crear_dashboard_ventas(ventas, productos, clientes)

## 8. Exportaci√≥n de Gr√°ficos

In [ ]:
# Crear directorio de salida si no existe
import os
output_dir = '../../outputs/visualizaciones'
os.makedirs(output_dir, exist_ok=True)

# Ejemplo: Guardar gr√°fico en alta resoluci√≥n
plt.figure(figsize=(12, 6))
ventas_categoria = ventas_productos.groupby('categoria')['total'].sum().sort_values()
plt.barh(ventas_categoria.index, ventas_categoria.values, color='#2E86AB')
plt.title('Ventas por Categor√≠a', fontsize=16, fontweight='bold')
plt.xlabel('Total Ventas ($)')
plt.tight_layout()

# Guardar en diferentes formatos
plt.savefig(f'{output_dir}/ventas_categoria.png', dpi=300, bbox_inches='tight')
plt.savefig(f'{output_dir}/ventas_categoria.pdf', bbox_inches='tight')
plt.savefig(f'{output_dir}/ventas_categoria.svg', bbox_inches='tight')
plt.show()

print(f"‚úÖ Gr√°ficos exportados a: {output_dir}")
print("üìÅ Formatos: PNG (300 DPI), PDF, SVG")

## üéØ Ejercicios Pr√°cticos

### Ejercicio 1: Dashboard Personalizado
Crea un dashboard con los siguientes gr√°ficos:
1. Histograma de distribuci√≥n de edades de clientes
2. Box plot de ventas por pa√≠s
3. Heatmap de ventas por d√≠a de la semana y hora
4. Sunburst de categor√≠a ‚Üí producto ‚Üí m√©todo de pago

In [ ]:
# TU C√ìDIGO AQU√ç

### Ejercicio 2: An√°lisis Temporal
Crea una visualizaci√≥n que muestre:
- Ventas acumuladas a lo largo del tiempo
- Promedio m√≥vil de 7 d√≠as
- Identificar tendencias alcistas/bajistas

In [ ]:
# TU C√ìDIGO AQU√ç

### Ejercicio 3: Visualizaci√≥n Interactiva
Usa Plotly para crear un dashboard interactivo con:
- Filtros por categor√≠a y rango de fechas
- Gr√°ficos que se actualicen din√°micamente
- Tooltips personalizados con informaci√≥n relevante

In [ ]:
# TU C√ìDIGO AQU√ç

## üìö Recursos Adicionales

### Documentaci√≥n
- [Matplotlib Documentation](https://matplotlib.org/stable/contents.html)
- [Seaborn Gallery](https://seaborn.pydata.org/examples/index.html)
- [Plotly Python](https://plotly.com/python/)

### Inspiraci√≥n
- [From Data to Viz](https://www.data-to-viz.com/)
- [Python Graph Gallery](https://www.python-graph-gallery.com/)
- [Storytelling with Data](http://www.storytellingwithdata.com/)

### Librer√≠as Avanzadas
- **Altair**: Visualizaci√≥n declarativa
- **Bokeh**: Gr√°ficos interactivos para web
- **Plotnine**: Gr√°ficos estilo ggplot2
- **Dash**: Frameworks para dashboards

---

## ‚úÖ Resumen

En este notebook aprendiste:

1. ‚úÖ **Matplotlib**: Gr√°ficos est√°ticos personalizables (l√≠neas, barras, pie, subplots)
2. ‚úÖ **Seaborn**: Visualizaciones estad√≠sticas (boxplot, violin, heatmap, pairplot)
3. ‚úÖ **Plotly**: Gr√°ficos interactivos para dashboards web
4. ‚úÖ **Mejores pr√°cticas**: Claridad, colores, tipos de gr√°ficos, anotaciones
5. ‚úÖ **Dashboards**: Combinar m√∫ltiples visualizaciones
6. ‚úÖ **Exportaci√≥n**: Guardar gr√°ficos en alta calidad

**üéØ Pr√≥ximo paso**: Control de versiones con Git

---